In [1]:
import numpy as np
import copy as cp
import random
from graphics import *
from numpy import *
from datetime import datetime, date, time

import cmath

import sys

import math
import pylab
from matplotlib import mlab
import matplotlib.pyplot as plt


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
class tree:
    def __init__(self, data = 0, N = None, l = None, lm = None, rm = None, r = None, flag = 0):
        self.flag = flag
        self.data = data
        self.N = N
        self.l  = l 
        self.lm = lm
        self.rm = rm
        self.r  = r
        

    def create_tree (self, x, y): 
        n0=0;n1=0;n2=0;n3=0
        n_0 = [];n_1 = [];n_2 = [];n_3 = []

        self.l = None
        self.lm = None
        self.rm = None
        self.r = None

        if self.N[1] > 1:    
            xm = self.N[2][0]
            ym = self.N[2][1]
            
            for j in range (0,self.N[1]):
                i = self.data[j]
                if x[i] < xm:    
                    if y[i] < ym:
                        n_0.append(i)
                        n0=n0+1
                    else:
                        n_1.append(i)
                        n1=n1+1
                else:
                    if y[i] >= ym:
                        n_2.append(i)
                        n2=n2+1
                    else:
                        n_3.append(i)
                        n3=n3+1
                        
            if n0>0:
                self.l = tree(n_0,[0+self.N[0], n0, None, None])
            if n1>0:
                self.lm = tree(n_1, [n0+self.N[0], n1, None, None])
            if n2>0:
                self.rm = tree(n_2, [n0+n1+self.N[0], n2, None, None])
            if n3>0:
                self.r = tree(n_3, [n0+n1+n2+self.N[0], n3, None, None])



In [3]:
def mid (t, x,y):    
    if t.N[2] == None:
        t.N[2] = [0,0]            
        for i in range(0, t.N[1]):
            t.N[2][0] = t.N[2][0] + x[t.data[i]]
            t.N[2][1] = t.N[2][1] + y[t.data[i]]
        t.N[2][0] = t.N[2][0]/t.N[1]
        t.N[2][1] = t.N[2][1]/t.N[1]
        
    if (t.N[3] == None):
        xmax = x[t.data[0]]; xmin = x[t.data[0]]; ymax = y[t.data[0]]; ymin = y[t.data[0]];
        for j in range (0,t.N[1]):
            i = t.data[j]
            xmax = max(x[i],xmax)
            ymax = max(y[i],ymax)
            xmin = min(x[i],xmin)
            ymin = min(y[i],ymin)
        t.N[3] = [xmax, ymax, xmin, ymin]
    return t.N
     


In [74]:
def count_svd (R, V):
    global k
    global eps
    B = np.zeros((R.N[1], V.N[1]), dtype = complex)
    for i in range (0, R.N[1]):
        for j in range (0, V.N[1]):
            lenght = ((x1[R.data[i]]-x2[V.data[j]])**2+(y1[R.data[i]]-y2[V.data[j]])**2)**0.5
           # interaction = cmath.exp(complex(0,1)*5*lenght)/lenght
            if lenght <> 0:
                B[i][j] = cmath.exp(complex(0,1)*k*lenght)/lenght
            else:
                B[i][j] = 0
                
    U,s,V = np.linalg.svd(B)


    summ_s = sum(s)
    summ_1 = 0.

#    l = len(s)-1
#    for i in range (0, len(s)):
#        if summ_1/summ_s<=1 - eps:
#            l = i
#            summ_1 = summ_1 +s[-l]
################

    #l = len(s)-1
    rank = len(s)
    for i in xrange(len(s)):
        if s[i]/s[0] <=  eps:
            #l = i
            rank = i + 1
            break
    s = s[0:rank]
    S = np.zeros((s.shape[0], s.shape[0]), complex)
    S[:s.shape[0], :s.shape[0]] = np.diag(s)

    U1 = np.zeros((U.shape[0], rank), complex)
    for i in range(0,U.shape[0]):
        for j in range (0,rank):
            U1[i][j]=U[i][j]
    U = U1
    V = V[0:rank][:]
#####################
#    s = s[0:l+1]

#    U1 = np.zeros((U.shape[0], l+1), complex)
#    for i in range(0,U.shape[0]):
#        for j in range (0,l+1):
#            U1[i][j]=U[i][j]
#    U = U1
#    V = V[0:l+1][:]

    #print np.linalg.matrix_rank(U)
    
    return U,s,V
    
        


In [87]:
def pre_count_A (R, V):
    
    global d
    global C0
    global k
    
    R.N = mid(R, x1, y1)
    V.N = mid(V, x2, y2)

    lenghtR = ((R.N[3][0] - R.N[3][2])**2 + (R.N[3][1] - R.N[3][3])**2)**0.5
    lenghtV = ((V.N[3][0] - V.N[3][2])**2 + (V.N[3][1] - V.N[3][3])**2)**0.5

    lenght = ((R.N[2][0] - V.N[2][0])**2 + (R.N[2][1] - V.N[2][1])**2)**0.5
    
    d = max(lenghtR, lenghtV)

    if (max(d*k/(2*pi), C0)*d>=lenght):
    #if d>=lenght:
        if(R.l==None)&(R.lm==None)&(R.rm==None)&(R.r==None):
            R.create_tree(x1, y1)
        if(V.l==None)&(V.lm==None)&(V.rm==None)&(V.r==None):
            V.create_tree(x2, y2)

        n_R_0 =[]; n_R_1 =[]; n_R_2 =[]; n_R_3 =[]; n_V_0 =[]; n_V_1 =[]; n_V_2 =[]; n_V_3 =[]
        
        
        if (R.N[1]>1):
            if (V.N[1]>1):
                p = 0#k#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                if (R.l<>None):
                    if (V.l<>None):
                        pre_count_A(R.l, V.l)
                    if (V.lm<>None):
                        pre_count_A(R.l, V.lm)
                    if (V.rm<>None):
                        pre_count_A(R.l, V.rm)
                    if (V.r<>None):
                        pre_count_A(R.l, V.r)



                if (R.lm<>None):
                    if (V.l<>None):
                        pre_count_A(R.lm, V.l)
                    if (V.lm<>None):
                        pre_count_A(R.lm, V.lm)
                    if (V.rm<>None):
                        pre_count_A(R.lm, V.rm)
                    if (V.r<>None):
                        pre_count_A(R.lm, V.r)
                  
                    
                if (R.rm<>None):
                    if (V.l<>None):
                        pre_count_A(R.rm, V.l)
                    if (V.lm<>None):
                        pre_count_A(R.rm, V.lm)
                    if (V.rm<>None):
                        pre_count_A(R.rm, V.rm)
                    if (V.r<>None):
                        pre_count_A(R.rm, V.r)
                   
                    
                if (R.r<>None):
                    if (V.l<>None):
                        pre_count_A(R.r, V.l)
                    if (V.lm<>None):
                        pre_count_A(R.r, V.lm)
                    if (V.rm<>None):
                        pre_count_A(R.r, V.rm)
                    if (V.r<>None):
                        pre_count_A(R.r, V.r)
                    
            else:
               
                if (R.l<>None):
                   pre_count_A(R.l, V)
                if (R.lm<>None):
                   pre_count_A(R.lm, V)
                if (R.rm<>None):
                   pre_count_A(R.rm, V)
                if (R.r<>None):
                   pre_count_A(R.r, V)
                
        else:
            if (V.N[1]>1):
                  
                if (V.l<>None):
                   pre_count_A(R, V.l)
                if (V.lm<>None):
                   pre_count_A(R, V.lm)
                if (V.rm<>None):
                   pre_count_A(R, V.rm)
                if (V.r<>None):
                   pre_count_A(R, V.r)
                
            
        flag = 0
        if R.flag > -1:
            if (R.l<>None):
                n_R_0 = R.l.data
                flag = flag + 1
            if (R.lm<>None):
                n_R_1 = R.lm.data
                flag = flag + 1
            if (R.rm<>None):
                n_R_2 = R.rm.data
                flag = flag + 1
            if (R.r<>None):
                n_R_3 = R.r.data
                flag = flag + 1
            if flag > 0:
                R.data = n_R_0 + n_R_1 + n_R_2 + n_R_3
            R.flag = R.flag + 1
        
        
        flag = 0
        if V.flag > -1:
            if (V.l<>None):
                n_V_0 = V.l.data
                flag = flag + 1
            if (V.lm<>None):
                n_V_1 = V.lm.data
                flag = flag + 1
            if (V.rm<>None):
                n_V_2 = V.rm.data
                flag = flag + 1
            if (V.r<>None):
                n_V_3 = V.r.data
                flag = flag + 1
            if flag > 0:
                V.data = n_V_0 + n_V_1 + n_V_2 + n_V_3
            V.flag = V.flag + 1
  


In [88]:
#%matplotlib inline
def count_A (R, V):
    
    global k

    R.N = mid(R, x1, y1)
    V.N = mid(V, x2, y2)

    lenghtR = ((R.N[3][0] - R.N[3][2])**2 + (R.N[3][1] - R.N[3][3])**2)**0.5
    lenghtV = ((V.N[3][0] - V.N[3][2])**2 + (V.N[3][1] - V.N[3][3])**2)**0.5

    lenght = ((R.N[2][0] - V.N[2][0])**2 + (R.N[2][1] - V.N[2][1])**2)**0.5
    
    d = max(lenghtR, lenghtV)

    if (max(d*k/(2*pi),C0)*d <lenght):
    #if d<lenght:    
        
        A = [count_svd(R, V)]
    else:
        A = []        
        
        if (R.N[1]>1):
            if (V.N[1]>1):
                p = 0
                if (R.l<>None):
                    A = A+[[]]
                    if (V.l<>None):
                        A[p] = A[p] + [count_A(R.l, V.l)]
                    if (V.lm<>None):
                        A[p] = A[p] + [count_A(R.l, V.lm)]
                    if (V.rm<>None):
                        A[p] = A[p] + [count_A(R.l, V.rm)]
                    if (V.r<>None):
                        A[p] = A[p] + [count_A(R.l, V.r)]
                    p = p+1


                if (R.lm<>None):
                    A = A + [[]]
                    if (V.l<>None):
                       A[p] = A[p] + [count_A(R.lm, V.l)]
                    if (V.lm<>None):
                       A[p] = A[p] + [count_A(R.lm, V.lm)]
                    if (V.rm<>None):
                       A[p] = A[p] + [count_A(R.lm, V.rm)]
                    if (V.r<>None):
                       A[p] = A[p] + [count_A(R.lm, V.r)]
                    p = p+1
                    
                if (R.rm<>None):
                    A = A + [[]]
                    if (V.l<>None):
                       A[p] = A[p] + [count_A(R.rm, V.l)]
                    if (V.lm<>None):
                       A[p] = A[p] + [count_A(R.rm, V.lm)]
                    if (V.rm<>None):
                       A[p] = A[p] + [count_A(R.rm, V.rm)]
                    if (V.r<>None):
                       A[p] = A[p] + [count_A(R.rm, V.r)]
                    p = p+1
                    
                if (R.r<>None):
                    A = A + [[]]
                    if (V.l<>None):
                       A[p] = A[p] + [count_A(R.r, V.l)]
                    if (V.lm<>None):
                       A[p] = A[p] + [count_A(R.r, V.lm)]
                    if (V.rm<>None):
                       A[p] = A[p] + [count_A(R.r, V.rm)]
                    if (V.r<>None):
                       A[p] = A[p] + [count_A(R.r, V.r)]
                    
            else:
                A = A + []
                if (R.l<>None):
                   A = A + [[count_A(R.l, V)]]
                if (R.lm<>None):
                   A = A + [[count_A(R.lm, V)]]
                if (R.rm<>None):
                   A = A + [[count_A(R.rm, V)]]
                if (R.r<>None):
                   A = A + [[count_A(R.r, V)]]
                
        else:
            if (V.N[1]>1):
                A = A + [[]]    
                if (V.l<>None):
                   A[0] = A[0] + [count_A(R, V.l)]
                if (V.lm<>None):
                   A[0] = A[0] + [count_A(R, V.lm)]
                if (V.rm<>None):
                   A[0] = A[0] + [count_A(R, V.rm)]
                if (V.r<>None):
                   A[0] = A[0] + [count_A(R, V.r)]
                
            else:
                A = [[0]]
              
    return A


In [77]:
def summ(a,b):
    res = []
    for i in range(0, len(a)):
        res.append(a[i]+b[i])
    return res


In [78]:
def dot_Mx(A, x, w):
    M = cp.deepcopy(A)
    if (type(M) == tuple):
        U, s, V = M
        S = np.zeros((U.shape[1], V.shape[0]), complex)
        S[:s.shape[0], :s.shape[0]] = np.diag(s)
        n = V.shape[1]
        f = np.dot(np.dot(np.dot(U,S),V),x[w:w+n]).tolist()
    if M == 0:
        f = [0]
        n = 1
    if type(M) == list:
        if M == [0]:
            f = [0]
            n = 1
        if M == []:
            f = []
            n = 0
        else:            
            string = M.pop(0)
            M3 = M
            if (len(string)<>0)and(type(string)<>tuple):
                M1 = string.pop(0)
                M2 = [string]
            else:
                M1 = string
                M2 = []
            (f1, n1) = dot_Mx(M1, x, w)
            (f2, n2) = dot_Mx(M2, x, w+n1)
            (f3, n3) = dot_Mx(M3, x, w)
            
            
            if f2 <> []:
                f = summ(f1, f2)+f3
            else:
                f = f1+f3
            n = n1+n2
    return f, n


In [79]:
def final_dot (x):
    x_new = []
    f_true = []
    for i in range(0,nv):
        x_new.append(x[V.data[i]])
        
    f = dot_Mx(A,x_new,0)

    for i in range (0, nr):
        f_true.append(0)
    for i in range (0, nr):
        f_true[R.data[i]] = f[0][i]
    
    #print size(x_new)
    
    return f_true

def final_dot_r (x):
    x_new = []
    f_true = []
    for i in range(0,nv):
        x_new.append(x[R.data[i]])
        
    f = dot_Mx(A_r,x_new,0)

    for i in range (0, nr):
        f_true.append(0)
    for i in range (0, nr):
        f_true[V.data[i]] = f[0][i]
    
    #print size(x_new)
    
    return f_true

In [10]:

k = 0.
eps = 0.1
nr = 1000
nv = 1000



x1=[]
y1=[]
x2=[]
y2=[]
for i in range (0,nr):
    #x1.append(random.randint(10,20))
    #y1.append(random.randint(10,20))
    x1.append(random.uniform(0,1))
    y1.append(random.uniform(0,1))
for i in range (0,nv):
    #x2.append(random.randint(0,1))
    #y2.append(random.randint(0,1))
    x2.append(random.uniform(0,1))
    y2.append(random.uniform(0,1))
    
#x2 = x1
#y2 = y1


In [25]:
C0 =1.

In [11]:
x=[]

for i in range (0,nv):
    x.append(i)


In [80]:
print k, C0, eps

500 2 0.1


In [93]:
k=1
eps=0.1
C0=2

In [94]:
#R - приёмник
#V - источник
t = datetime.today()
n_r  = []
n_v  = []
for i in range (0,nr):
    n_r.append(i)
for i in range (0,nv):
    n_v.append(i)


R = tree(n_r,[0,nr, None, None])
V = tree(n_v,[0,nv, None, None])
pre_count_A(R,V)
A = count_A(R,V)
#A_r = count_A(V,R)
"ok"
print datetime.today() - t
#print (A[0][0])

print k

0:00:07.020065
1


In [204]:
x0 = 650; y0 = 650




def draw_strings(A, x, y, size_x, size_y):
    if (type(A)==list):
        n = len(A)
        if n<>0:
            hight = size_y/n
            for i in range (1, n):
                string = Line(Point(x, y+i*hight), Point(x+size_x, y+i*hight))
                string.setWidth(1)
                string.draw(win)
            for j in range (0, n):
                draw_columns(A[j], x, y+j*hight, size_x, hight)
    else:
        if A == 0:
            rectangle = Rectangle(Point(x, y), Point(x+size_x, y+size_y))
            rectangle.setFill('red')
            rectangle.draw(win)
        else:
            rectangle = Rectangle(Point(x, y), Point(x+size_x, y+size_y))
            rectangle.setFill('green')
            rectangle.draw(win)
            
    
    
def draw_columns(A, x, y, size_x, size_y):
    if type(A)==list:
        n = len(A)
        if n<>0:

            width = size_x/n
            for i in range (1, n):
                string = Line(Point(x+i*width, y), Point(x+i*width, y+size_y))
                string.setWidth(1)
                string.draw(win)
            for j in range (0, n):
                draw_strings(A[j], x+j*width, y, width, size_y)
    else:
        if A == 0:
            rectangle = Rectangle(Point(x, y), Point(x+size_x, y+size_y))
            rectangle.setFill('red')
            rectangle.draw(win)
        else:
            rectangle = Rectangle(Point(x, y), Point(x+size_x, y+size_y))
            rectangle.setFill('green')
            rectangle.draw(win)

        
win = GraphWin('A', x0, y0)


draw_strings(A, 0., 0., 650.,650.)

win.getMouse()
win.close()

GraphicsError: Can't draw to closed window

In [16]:
def usual_count_A(x1, y1, x2, y2):
    #print k
    nr = len(x1)
    nv = len(x2)
    usual_A = np.zeros((nr,nv), complex)
    for i in range (0, nr):
        for j in range (0, nv):
            lenght = ((x1[i]-x2[j])**2+(y1[i]-y2[j])**2)**0.5
            if lenght <> 0:
                usual_A[i][j] = cmath.exp(complex(0,1)*k*lenght)/lenght
            else:
                usual_A[i][j] = 0

    return usual_A

In [17]:
def usual_dot_Ax(A,x):
    usual_f = np.dot(A,x)
        
    return usual_f

In [18]:
def norm_ff(f1, f2):
    norm_between_f = 0.
    norm_f = 0.
    for i in range (0,nr):
        norm_between_f = norm_between_f+(abs(f1[i]-f2[i]))**2
        norm_f = norm_f+abs(f2[i]**2)
    norm_f = (norm_f)**0.5
    norm_between_f = (norm_between_f**0.5)/norm_f
    return norm_between_f

In [33]:
mat = np.ndarray((nr, nv), np.complex128)
for i in range(nr):
    mat[:,i] = final_dot(np.eye(nr)[:,i])
#print mat

#f1 = final_dot(x)
#print f1
#np.linalg.svd(mat)[1]
#print usual_A, mat

KeyboardInterrupt: 

In [91]:
t = datetime.today()
usual_A = usual_count_A(x1,y1,x2,y2)
usual_f = usual_dot_Ax(usual_A,x)
f = final_dot(x)
norm_between_f = norm_ff(usual_f, f)
print datetime.today()-t
print norm_between_f
print size(f)
print k, C0

#print np.linalg.matrix_rank(mat)
#print np.linalg.svd(usual_A-mat)[1][0]
#print np.linalg.svd(mat)[1][0]
#print np.linalg.svd(np.linalg.inv(mat))[1][0]
#print np.linalg.svd(usual_A-mat)[1][0]*np.linalg.svd(np.linalg.inv(mat))[1][0]
#print np.linalg.svd(usual_A-mat)[1][0]/np.linalg.svd(usual_A)[1][0]

0:00:22.169631
0.00157878739333
1000
10 2


In [1]:
#from pypropack import svdp
#import scipy.sparse.linalg as la

#linop_diff = la.LinearOperator((nr, nv), matvec = lambda x: final_dot(x)-usual_A.dot(x), rmatvec = lambda x: final_dot_r(x)-usual_A.T.dot(x), dtype = np.complex128)
#linop_A = la.LinearOperator((nr, nv), matvec = lambda x: usual_A.dot(x), rmatvec = lambda x: usual_A.T.dot(x), dtype = np.complex128)
#s_diff = svdp(linop_diff, 1, compute_u=0, compute_v=0,kmax=100)
#s_A = svdp(linop_A, 1, compute_u=0, compute_v=0,kmax=100)
#print s_diff[0]/s_A[0]

In [26]:
#svdp?
#import numpy as np
#from pypropack import svdp

#A = np.random.randn(30, 30, 2).view(dtype=np.complex128)[:,:,0]

#u, sigma, vt = svdp(A, 1, kmax = 100)

In [25]:
import sys
def show_sizeof(x, level=0):

    #print "\t" * level, x.__class__, sys.getsizeof(x), x
    #res = sys.getsizeof(x)
    res = 0.
    if hasattr(x, '__iter__'):
        #if hasattr(x, 'items'):
        if isinstance(x, np.ndarray):
            #for xx in x.items():
                #res += show_sizeof(xx, level + 1)
            res += x.nbytes
        else:
            for xx in x:
                res += show_sizeof(xx, level + 1)
    return res

In [95]:
print show_sizeof(A), show_sizeof(usual_A)
#usual_A = usual_count_A(x1,y1,x2,y2)
print  np.linalg.matrix_rank(usual_A),k


7069328.0 16000000.0
989 1


In [73]:
k=1
eps=0.01
C0 = 2

In [71]:
print k, C0

1 4


In [74]:
def character_of_A(n):
    
    def final_dot (x):
        x_new = []
        f_true = []
        for i in range(0,nv):
            x_new.append(x[V.data[i]])

        f = dot_Mx(A,x_new,0)

        for i in range (0, nr):
            f_true.append(0)
        for i in range (0, nr):
            f_true[R.data[i]] = f[0][i]

        #print size(x_new)

        return f_true
    
    
    global nr 
    global nv 
    global x1
    global y1
    global x2
    global y2
    global eps
    global k
    nr = n
    nv = n
    x1=[]
    y1=[]
    x2=[]
    y2=[]
    for i in range (0,nr):
        x1.append(random.uniform(0,1))
        y1.append(random.uniform(0,1))
    for i in range (0,nv):
        x2.append(random.uniform(0,1))
        y2.append(random.uniform(0,1))

    #x2 = x1
    #y2 = y1
    
    x=[]

    for i in range (0,nv):
        x.append(i)
        
    n_r  = []
    n_v  = []
    for i in range (0,nr):
        n_r.append(i)
    for i in range (0,nv):
        n_v.append(i)
    
#    k=300

    R = tree(n_r,[0,nr, None, None])
    V = tree(n_v,[0,nv, None, None])
    pre_count_A(R,V)
    A = count_A(R,V)
    usual_A = usual_count_A(x1,y1,x2,y2)
#    print [show_sizeof(A),show_sizeof(usual_A)]
    usual_f = usual_dot_Ax(usual_A,x)
    f = final_dot(x)
#    norm_between_f = 
#    print n, size(x)
    return [show_sizeof(A),show_sizeof(usual_A), norm_ff(usual_f, f), np.linalg.matrix_rank(usual_A)]


print size(x)

log_n_list = []
m = 10
for i in range (1, m+1):
    log_n_list.append(2**i) 

#k=300
    
log_n_list1 = []
l = 14
for i in range (0, l):
    log_n_list1.append(2**(-i)) 

print log_n_list1

ylist = [character_of_A(n) for n in log_n_list]

#print ylist[1][0]
#ylist2 = [character_of_A(2**n)[1] for n in log_n_list]


plt.figure(1)
plt.title('Amount of informaton')
plt.xlabel('n')
plt.ylabel('Bytes')
plt.plot (log_n_list, [ylist[i][0] for i in range (0,m)])
plt.plot (log_n_list, [ylist[i][1] for i in range (0,m)])
plt.figure(2)
plt.title('delta f/|f|')
plt.xlabel('n')
plt.ylabel('delta f/|f|')

plt.plot (log_n_list, [ylist[i][2] for i in range (0,m)])
print eps
#eps = 1.
#plt.plot (log_n_list, [ylist[i][2] for i in range (0,m)])
plt.figure(3)
plt.title('Rank usual A')
plt.xlabel('n')
plt.ylabel('Rank')
plt.plot (log_n_list, [ylist[i][3] for i in range (0,m)])

#pylab.plot (log_n_list, ylist2)


plt.show()
print 'k=', k, C0

100
[1, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625, 0.001953125, 0.0009765625, 0.00048828125, 0.000244140625, 0.0001220703125]
0.01
k= 1 2


In [ ]:
len(log_n_list)

In [24]:
u = 2+3j
u
s = complex(0,1)
cmath.exp(s*pi)

(-1+1.2246467991473532e-16j)

In [94]:
k = 5.
r = 0.5
g = cmath.exp(complex(0,1)*k*r)/r
#exp**(s*pi)

In [95]:
type(g)

complex

In [105]:
a = [0.+0.j, 0.+0.j]
a[1]=g
a
real(a[1])

array(-1.6022872310938674)

In [21]:
eps

0.001